In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import sampler

import numpy as np
from numpy import vstack
from numpy import sqrt
import pandas as pd
from pandas import read_csv
import pickle as pkl

import io as b_io

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 1000
print('using device:', device)

using device: cpu


In [3]:
# Load feature vectors
out_file = "./data/CONTUS_POP_self_storage.pkl"
with open(out_file, "rb") as f:
    data = np.load(f, allow_pickle = True)
print(len(data["X"]))
print(type(data["X"]))
print(len(data["latlon"]))
temp = b_io.BytesIO(data["X"])
features = np.load(temp, allow_pickle=True)
print(features.shape)
latlons_samp = data["latlon"]
ids_x = data["ids_X"]

X = pd.DataFrame(features, index = ids_x)
X.index.name = "foo"
X = X.sort_values("foo", ascending=True)
lls = pd.DataFrame(latlons_samp, index=ids_x, columns=["lat", "lon"])

75235456
<class 'bytes'>
2296
(2296, 8192)


In [4]:
# Load labels
data_train=pd.read_csv('./data/train_df_with_census_data.tsv',sep='\t')
data_test=pd.read_csv('./data/test_id_df_with_census_data.tsv',sep='\t')
data_test2=pd.read_csv('./data/test_ood_df_with_census_data.tsv',sep='\t')
labels_dataset = [data_train, data_test, data_test2]
y_dataset = pd.concat(labels_dataset).sort_values("places_id", ascending=True)
y_labels = y_dataset.groupby(["places_id"]).mean()
y = y_labels["price"]
# Normalize Inputs
# y = (y_labels["price"] - y_labels["price"].mean())/y_labels["price"].std()

In [5]:
# Split dataset
NUM_EXAMPLES = len(X)
NUM_TRAIN = int(NUM_EXAMPLES * 0.8)
NUM_VAL = (NUM_EXAMPLES - NUM_TRAIN) // 2
NUM_TEST = NUM_EXAMPLES - NUM_VAL - NUM_TRAIN

X_train_np = X[0:NUM_TRAIN].values
X_val_np = X[NUM_TRAIN:NUM_TRAIN + NUM_VAL].values
X_test_np = X[NUM_TRAIN + NUM_VAL:].values
X_train = torch.from_numpy(X_train_np.astype(np.float32))
X_val = torch.from_numpy(X_val_np.astype(np.float32))
X_test = torch.from_numpy(X_test_np.astype(np.float32))

y_train_np = y[0:NUM_TRAIN].values
y_val_np = y[NUM_TRAIN:NUM_TRAIN + NUM_VAL].values
y_test_np = y[NUM_TRAIN + NUM_VAL:].values
y_train = torch.from_numpy(y_train_np.astype(np.float32))
y_val = torch.from_numpy(y_val_np.astype(np.float32))
y_test = torch.from_numpy(y_test_np.astype(np.float32))

train_dataset = TensorDataset(X_train, y_train) 
loader_train = DataLoader(train_dataset, batch_size=68)

val_dataset = TensorDataset(X_val, y_val) 
loader_val = DataLoader(val_dataset, batch_size=46)

test_dataset = TensorDataset(X_test, y_test) 
loader_test = DataLoader(test_dataset, batch_size=46)

In [6]:
# Define Model
no_input_features = X_train_np.shape[1]
class Three_Layer_Network(torch.nn.Module):
    def __init__(self,no_input_features):
        super(Three_Layer_Network,self).__init__()
        self.layer1=nn.Linear(no_input_features,10)
        nn.init.xavier_uniform_(self.layer1.weight)
        self.layer2=nn.ReLU()
        self.layer3=nn.Linear(10,8)
        nn.init.xavier_uniform_(self.layer3.weight)
        self.layer4=nn.ReLU()
        self.layer5=nn.Linear(8,1)
        nn.init.xavier_uniform_(self.layer5.weight)
    def forward(self,x):
       y_predicted=self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x)))))
       return y_predicted

In [7]:
# Define Evaluation Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    r2 = r2_score(actuals, predictions)
    return mse, r2

In [8]:
def train_model(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device)  # move to device, e.g. GPU
            y = y.to(device=device)
            
            criterion = nn.MSELoss()
            scores = model(x)
            loss = criterion(torch.reshape(scores, (68,)), y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()
            
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                val_loss, r2 = evaluate_model(loader_val, model)
                print('Validation loss = %.4f' % (val_loss))

In [ ]:
model= Three_Layer_Network(no_input_features)
learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=2e-5)
train_model(model, optimizer, 1000)

Iteration 0, loss = 44211.5039
Validation loss = 34997.4141
Iteration 0, loss = 6481.4214
Validation loss = 6973.7227
Iteration 0, loss = 5792.6948
Validation loss = 6685.8555
Iteration 0, loss = 5729.4678
Validation loss = 6606.5068
Iteration 0, loss = 5698.3262
Validation loss = 6576.1631
Iteration 0, loss = 5652.6265
Validation loss = 6546.0400
Iteration 0, loss = 5609.7915
Validation loss = 6520.4141
Iteration 0, loss = 5575.4092
Validation loss = 6499.6499
Iteration 0, loss = 5548.3540
Validation loss = 6482.6650
Iteration 0, loss = 5525.8584
Validation loss = 6468.4277
Iteration 0, loss = 5506.3281
Validation loss = 6456.4937
Iteration 0, loss = 5489.5488
Validation loss = 6446.8022
Iteration 0, loss = 5475.4805
Validation loss = 6439.1528
Iteration 0, loss = 5463.7925
Validation loss = 6433.1816
Iteration 0, loss = 5454.0698
Validation loss = 6428.5156
Iteration 0, loss = 5445.9146
Validation loss = 6424.8306
Iteration 0, loss = 5438.9648
Validation loss = 6421.8682
Iteration 0,

In [ ]:
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
mse, r2 = evaluate_model(loader_test, model)

In [ ]:
mse

In [ ]:
r2

In [ ]:
# Define Model
no_input_features = X_train_np.shape[1]
class Five_Layer_Network(torch.nn.Module):
    def __init__(self,no_input_features):
        super(Three_Layer_Network,self).__init__()
        self.layer1=nn.Linear(no_input_features, 30)
        nn.init.xavier_uniform_(self.layer1.weight)
        self.layer2=nn.ReLU()
        self.layer3=nn.Linear(30,10)
        nn.init.xavier_uniform_(self.layer3.weight)
        self.layer4=nn.ReLU()
        self.layer5=nn.Linear(10,8)
        nn.init.xavier_uniform_(self.layer5.weight)
        self.layer6=nn.ReLU()
        self.layer7=nn.Linear(8,1)
        nn.init.xavier_uniform_(self.layer7.weight)
    def forward(self,x):
       y_predicted=self.layer7(self.layer6(self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x)))))))
       return y_predicted

In [ ]:
model= Three_Layer_Network(no_input_features)
learning_rate = 1e-2
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=2e-5)
train_model(model, optimizer, 1000)

In [ ]:
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
mse, r2 = evaluate_model(loader_test, model)

In [ ]:
mse

In [ ]:
r2